#### Turtle Trading

#### Data, Library Load

In [1]:
from tqdm import tqdm 
import pandas as pd
from copy import deepcopy
from dateutil.parser import parse
import datetime as dt
from pykrx import stock
import numpy as np

In [2]:
import sys
sys.path.append('D:/ML/Kis_Api_test/')
from BackTesting_System.backtesting import BackTesting,Utils

In [3]:
path = '../../../mom.csv'
path = '../../../df_230323_fin.csv'
df = pd.read_csv(path,index_col=0)
df = df.drop(df.iloc[:,8:],axis=1)
df.head(1)

C:\Users\SCH\AppData\Local\Temp\ipykernel_8612\1189732665.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path,index_col=0)


,날짜,종가,시가,최고가,최저가,누적거래량,종목명,종목코드
65,2018-01-02,11796.0,11971.0,11971.0,11470.0,128805.0,힘스,238490


In [4]:
df.shape

(2909257, 8)

In [5]:
df = df[df.apply(lambda x: '스팩' not in x['종목명'], axis=1)]
# df = df[df.apply(lambda x: '대성홀딩스' not in x['종목명'], axis=1)]
df.shape

(2892825, 8)

#### function define

In [6]:
# make momentum (10일,20일)
def make_mom(df):
    df = deepcopy(df)
    buf = []
    for _, gdf in tqdm(df.groupby('종목명')):
        gdf['10_mom'] = gdf['종가'] / gdf['종가'].shift(11)*100
        gdf['20_mom'] = gdf['종가'] / gdf['종가'].shift(21)*100
        buf.append(gdf)
    del df
    res = pd.concat(buf)
    return res 

In [7]:
# make tr
def make_tr(df):
    df = deepcopy(df)
    buf = []
    for _, gdf in tqdm(df.groupby('종목명')):
        gdf['con1'] = gdf['최고가'] - gdf['종가'].shift(1)
        gdf['con2'] = gdf['최저가'] - gdf['종가'].shift(1)
        gdf['con1_per'] = (gdf['최고가'] - gdf['종가'].shift(1))/gdf['종가'].shift(1)*100
        gdf['con2_per'] = (gdf['최저가'] - gdf['종가'].shift(1))/gdf['종가'].shift(1)*100
        buf.append(gdf) 
    del df
    res = pd.concat(buf)
    res['con3'] = res['최고가'] - res['최저가']
    res['con3_per'] = ((res['최고가']-res['최저가'])/res['최저가']*100).abs()
    res['con1'] = res['con1'].abs()
    res['con2'] = res['con2'].abs()
    res['con1_per'] = res['con1_per'].abs()
    res['con2_per'] = res['con2_per'].abs()
    res['TR'] = res[['con1','con2','con3']].max(axis=1)
    res['TR_per'] = res[['con1_per','con2_per','con3_per']].max(axis=1)
    res = res.drop(columns=['con1', 'con2', 'con3'], axis=1)
    res = res.drop(columns=['con1_per','con2_per','con3_per'], axis=1)
    return res 

In [8]:
# make atr 21일
def make_atr(df):
    df = deepcopy(df)
    buf = []
    for _, gdf in tqdm(df.groupby('종목명')):
        gdf['ATR'] = gdf['TR'].rolling(21).mean()
        gdf['ATR_per'] = gdf['TR_per'].rolling(21).mean()
        gdf['ATR'].fillna(np.inf, inplace=True)
        gdf['ATR_per'].fillna(np.inf, inplace=True)
        buf.append(gdf)
    del df
    res = pd.concat(buf)
    res = res.drop(columns=['TR'], axis=1)
    return res 

In [9]:
def make_raking(df):
    df = deepcopy(df)
    buf = []
    for _,gdf in tqdm(df.groupby('날짜')):
        gdf['ranking'] = gdf['ATR'].rank(method='min', ascending=True)
        buf.append(gdf)
    del df
    res = pd.concat(buf)
    return res

In [10]:
# make buy
def make_buy(df):
    df = deepcopy(df)
    buf = []
    for _, gdf in tqdm(df.groupby('종목명')):
        gdf['buy1'] = (gdf['20_mom']>100)&(gdf['종가'].rolling(window=20, closed='left').max() < gdf['종가'])  #&(gdf['ranking'] <= 10)     
        #gdf['middle_buy'] = (gdf['종가'].rolling(window=55, closed='left').max() < gdf['종가'])
        #gdf['buy'] = gdf['short_buy'] | gdf['middle_buy']
        buf.append(gdf)
    del df
    res = pd.concat(buf)
    df = deepcopy(res)
    del buf
    buf = []
    for (_, buy), gdf in tqdm(df.groupby(['날짜', 'buy1'])):
        if buy == True:
            gdf['buy2'] = gdf['ATR_per'].rank(method='min', ascending=True) <= 10
        buf.append(gdf)
    res = pd.concat(buf)
    res['buy'] = res['buy1'] & res['buy2']
    res = res.drop(columns=['buy1', 'buy2'], axis=1)
    return res

In [11]:
# make sell
def make_sell(df):
    df = deepcopy(df)
    buf = []
    for _,gdf in tqdm(df.groupby('종목명')):
        con1 = (gdf['10_mom'] > 100)&(gdf['종가'].rolling(window=10,closed='left').min() > gdf['종가'])
        #con2 = (gdf['10_mom'] < 100)&(gdf['종가'].rolling(window=10,closed='left').max() < gdf['종가'])
        gdf['sell'] = con1
        buf.append(gdf)
    del df
    res = pd.concat(buf)

    return res 

#### Do!

In [12]:
df = make_mom(df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2502/2502 [00:02<00:00, 914.79it/s]


In [13]:
df = make_tr(df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2502/2502 [00:05<00:00, 442.07it/s]


In [14]:
df = make_atr(df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2502/2502 [00:03<00:00, 692.07it/s]


In [15]:
# df = make_raking(df)

In [16]:
df = make_buy(df)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 2555/2556 [00:01<00:00, 1544.40it/s]


In [17]:
df = make_sell(df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2502/2502 [00:02<00:00, 922.64it/s]


In [18]:
# code_kospi = pd.read_csv('../../SRC/current_kospi_codes.csv')
# code_kosdaq = pd.read_csv('../../SRC/current_kosdaq_codes.csv')
# code = pd.concat([code_kospi,code_kosdaq])
# code.rename(columns={'회사명':'종목명'},inplace=True)
# code['종목코드'] = code['종목코드'].astype('str')

# df_signal = df.merge(code,on='종목명',how='inner')
# df_signal.rename(columns={'종목코드':'코드'},inplace=True)
# df_signal['구매수량']=1

In [19]:
# df_signal['코드'] = df_signal['코드'].apply(lambda x: str(x).zfill(6))

In [20]:
bt = BackTesting()
ut = Utils()

In [21]:
df['코드'] = df['종목코드'].apply(lambda x: str(x).zfill(6))
df_signal = df

In [22]:
df[df['코드'] == '003670']['buy'].unique()

array([False])

In [23]:
df[(df['코드'] == '003670') & df['buy'] == True]

,날짜,종가,시가,최고가,최저가,누적거래량,종목명,종목코드,10_mom,20_mom,TR_per,ATR,ATR_per,buy,sell,코드


### 일정기간

In [59]:
for days in [92]:
    first_date = parse(sorted(df_signal['날짜'].unique())[0])
    last_date = parse(sorted(df_signal['날짜'].unique())[-1])
    term = dt.timedelta(days=days)
    term_bias_date = first_date
    while last_date - term_bias_date > term:        
        df_signal_test = df_signal[((df_signal['날짜'] >= str(term_bias_date)) & (df_signal['날짜'] <= str(term_bias_date + term)))]
        df_stock = stock.get_index_fundamental(str(term_bias_date)[:10], str(term_bias_date + term)[:10], '1001')
        bt.MONEY = 10000000
        first_monry = bt.MONEY
        bt.buy_sell(df=df_signal_test,buy='buy',sell='sell')
        bt.update_information()
        
        a = pd.DataFrame(bt.ASSET).T
        a['코드'] = a.index
        b= df_signal_test[df_signal_test['날짜'] == sorted(df_signal_test['날짜'].unique())[-1]][['코드', '종가']]
        df_join = a.merge(b,on='코드', how='inner')
        res = pd.DataFrame((df_join['종가'] - df_join['평단가'])  * df_join['수량']).sum()[0]
        c = df_join[['종가']].apply(lambda x: bt.calc_tax(x['종가'], '매도', True), axis=1).sum()
        
        print('-'*50)
        print('기간:' + str(term_bias_date) + '~' + str(term_bias_date + term))      
        print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')
        print('승리횟수:' + str(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0]))
        print('전체횟수:' + str(bt.ECONO_INFORM.shape[0]))
        print('평가손익:', round(res, 2) , ' ', '평가손익율:', round(res / (df_join['평단가']  * df_join['수량']).sum() * 100, 2))
        print('실현손익:', bt.ECONO_INFORM['실현손익'].sum())
        print('누적수익률', round(((bt.ECONO_INFORM['실현손익'].sum() + res - (bt.JOUNAL[bt.JOUNAL['매매구분']=='매수'][['매매비용']].sum()) - c).iloc[0])/ first_monry * 100, 2),'%')
        print('시장수익률:', + round(((df_stock.iloc[-1, 0] - df_stock.iloc[0, 0]) / df_stock.iloc[0, 0]) *100, 2), '%')
        print('CAGR:', round((((bt.MONEY + (df_join['종가'] * df_join['수량']).sum() - c) / first_monry) ** (1/1) - 1)*100, 2), '%')
        print('-'*50)
        term_bias_date = term_bias_date + term
    
    df_signal_test = df_signal[((df_signal['날짜'] >= str(term_bias_date)) & (df_signal['날짜'] <= str(last_date)))]
    df_stock = stock.get_index_fundamental(str(term_bias_date)[:10], str(last_date), '1001')
    bt.MONEY = 10000000
    first_monry = bt.MONEY
    bt.buy_sell(df=df_signal_test,buy='buy',sell='sell')
    bt.update_information()
    
    a = pd.DataFrame(bt.ASSET).T
    a['코드'] = a.index
    b= df_signal_test[df_signal_test['날짜'] == sorted(df_signal_test['날짜'].unique())[-1]][['코드', '종가']]
    df_join = a.merge(b,on='코드', how='inner')
    res = pd.DataFrame((df_join['종가'] - df_join['평단가'])  * df_join['수량']).sum()[0]
    c = df_join[['종가']].apply(lambda x: bt.calc_tax(x['종가'], '매도', True), axis=1).sum()
    
    print('-'*50)
    print('기간:' + str(term_bias_date) + '~' + str(last_date))
    print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')
    print('승리횟수:' + str(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0]))
    print('전체횟수:' + str(bt.ECONO_INFORM.shape[0]))
    print('평가손익:', round(res, 2) , ' ', '평가손익율:', round(res / (df_join['평단가']  * df_join['수량']).sum() * 100, 2))
    print('실현손익:', bt.ECONO_INFORM['실현손익'].sum())
    print('누적수익률', round(((bt.ECONO_INFORM['실현손익'].sum() + res - (bt.JOUNAL[bt.JOUNAL['매매구분']=='매수'][['매매비용']].sum()) - c).iloc[0])/ first_monry * 100, 2),'%')
    print('시장수익률:', + round(((df_stock.iloc[-1, 0] - df_stock.iloc[0, 0]) / df_stock.iloc[0, 0]) *100, 2), '%')
    print('CAGR:', round((((bt.MONEY + (df_join['종가'] * df_join['수량']).sum() - c) / first_monry) ** (1/1) - 1)*100, 2), '%')
    print()
    print('-'*50)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [00:06<00:00, 16.86it/s]


--------------------------------------------------
기간:2018-01-02 00:00:00~2018-04-04 00:00:00
승률: 9.3 %
승리횟수:4
전체횟수:43
평가손익: 331847.0   평가손익율: 3.45
실현손익: -302050.0
누적수익률 0.21 %
시장수익률: -2.89 %
CAGR: 0.21 %
--------------------------------------------------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 122/122 [00:06<00:00, 19.22it/s]


--------------------------------------------------
기간:2018-04-04 00:00:00~2018-07-05 00:00:00
승률: 11.67 %
승리횟수:7
전체횟수:60
평가손익: 188788.0   평가손익율: 2.03
실현손익: -95315.9999999998
누적수익률 0.81 %
시장수익률: -6.25 %
CAGR: 0.81 %
--------------------------------------------------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 122/122 [00:06<00:00, 18.95it/s]


...

--------------------------------------------------
기간:2022-07-16 00:00:00~2022-10-16 00:00:00
승률: 5.0 %
승리횟수:3
전체횟수:60
평가손익: 174116.0   평가손익율: 2.33
실현손익: -1101430.0000000002
누적수익률 -9.46 %
시장수익률: -6.85 %
CAGR: -9.46 %
--------------------------------------------------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 130/130 [00:08<00:00, 15.61it/s]


--------------------------------------------------
기간:2022-10-16 00:00:00~2023-01-16 00:00:00
승률: 15.87 %
승리횟수:10
전체횟수:63
평가손익: 805061.0   평가손익율: 8.42
실현손익: -254783.00000000003
누적수익률 5.28 %
시장수익률: 8.12 %
CAGR: 5.14 %
--------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:05<00:00, 15.31it/s]


--------------------------------------------------
기간:2023-01-16 00:00:00~2023-03-23 00:00:00
승률: 8.89 %
승리횟수:4
전체횟수:45
평가손익: 107751.0   평가손익율: 1.26
실현손익: -407029.0000000001
누적수익률 -3.21 %
시장수익률: 1.03 %
CAGR: -3.21 %

--------------------------------------------------


### 5년치 한꺼번에

In [24]:
first_date = parse(sorted(df_signal['날짜'].unique())[0])
last_date = parse(sorted(df_signal['날짜'].unique())[-1])
df_signal_test = df_signal
df_stock = stock.get_index_fundamental(str(first_date)[:10], str(last_date)[:10], '1001')
bt.MONEY = 10000000
first_monry = bt.MONEY
bt.buy_sell(df=df_signal_test,buy='buy',sell='sell', use_slippage=True)
bt.update_information()


a = pd.DataFrame(bt.ASSET).T
a['코드'] = a.index
b = df_signal[df_signal['날짜'] == sorted(df_signal_test['날짜'].unique())[-1]][['코드', '종가']]
df_join = a.merge(b,on='코드', how='inner')
res = pd.DataFrame((df_join['종가'] - df_join['평단가'])  * df_join['수량']).sum()[0]
c = df_join[['종가']].apply(lambda x: bt.calc_tax(x['종가'], '매도', True), axis=1).sum()

print('-'*50)
print('기간:' + str(first_date) + '~' + str(last_date))
print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')
print('승리횟수:' + str(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0]))
print('전체횟수:' + str(bt.ECONO_INFORM.shape[0]))
print('평가손익:', round(res, 2) , ' ', '평가손익율:', round(res / (df_join['평단가']  * df_join['수량']).sum() * 100, 2))
print('실현손익:', bt.ECONO_INFORM['실현손익'].sum())
print('누적수익률', round(((bt.ECONO_INFORM['실현손익'].sum() + res - (bt.JOUNAL[bt.JOUNAL['매매구분']=='매수'][['매매비용']].sum()) - c).iloc[0])/ first_monry * 100, 2),'%')
print('시장수익률:', + round(((df_stock.iloc[-1, 0] - df_stock.iloc[0, 0]) / df_stock.iloc[0, 0]) *100, 2), '%')
print('CAGR:', round((((bt.MONEY + (df_join['종가'] * df_join['수량']).sum() - c) / first_monry) ** (1/5) - 1)*100, 2), '%')
print('-'*50)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2555/2555 [02:44<00:00, 15.58it/s]


--------------------------------------------------
기간:2018-01-02 00:00:00~2023-03-23 00:00:00
승률: 21.27 %
승리횟수:262
전체횟수:1232
평가손익: 1860070.0   평가손익율: 9.05
실현손익: 11617994.0
누적수익률 134.01 %
시장수익률: -2.22 %
CAGR: 18.54 %
--------------------------------------------------


In [67]:
bt.JOUNAL[bt.JOUNAL['종목코드'] == '247540']

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분


In [61]:
# 마지막 유동성 + 계좌평가금액 - 계좌 청산시 비용
bt.MONEY + (df_join['종가']*df_join['수량']).sum() - c

23401345.0

In [62]:
# 처음 유동성 + 실현손익 + 평가손익 - 총매수비용 - 계좌 청산시 비용
first_monry + bt.ECONO_INFORM['실현손익'].sum() + res - (bt.JOUNAL[bt.JOUNAL['매매구분']=='매수'][['매매비용']].sum()) - c

매매비용    23401345.0
dtype: object

In [25]:
ut.making_mt_plot(handle_data=df_signal_test, code='003670', jounal=bt.JOUNAL)

### 원하는 날짜 기입

In [27]:
#######여기에 적어주세요##############
first_date = parse('2020-05-01')
last_date = parse('2020-08-01')
#####################################

df_signal_test = df_signal[((df_signal['날짜'] >= str(first_date)) & (df_signal['날짜'] <= str(last_date)))]
df_stock = stock.get_index_fundamental(str(first_date)[:10], str(last_date)[:10], '1001')
bt.MONEY = 10000000
first_monry = bt.MONEY
bt.buy_sell(df=df_signal_test,buy='buy',sell='sell')
bt.update_information()


a = pd.DataFrame(bt.ASSET).T
a['코드'] = a.index
b = df_signal[df_signal['날짜'] == sorted(df_signal_test['날짜'].unique())[-1]][['코드', '종가']]
df_join = a.merge(b,on='코드', how='inner')
res = pd.DataFrame((df_join['종가']* df_join['수량']) - (df_join['평단가']  * df_join['수량'])).sum()[0]
c = df_join[['종가']].apply(lambda x: bt.calc_tax(x['종가'], '매도', True), axis=1).sum()

print('-'*50)
print('기간:' + str(first_date) + '~' + str(last_date))
print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')
print('승리횟수:' + str(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0]))
print('전체횟수:' + str(bt.ECONO_INFORM.shape[0]))
print('평가손익:', round(res, 2) , ' ', '평가손익율:', round(res / (df_join['평단가']  * df_join['수량']).sum() * 100, 2))
print('실현손익:', bt.ECONO_INFORM['실현손익'].sum())
print('누적수익률', round(((bt.ECONO_INFORM['실현손익'].sum() + res - (bt.JOUNAL[bt.JOUNAL['매매구분']=='매수'][['매매비용']].sum()) - c).iloc[0])/ first_monry * 100, 2),'%')
print('시장수익률:', + round(((df_stock.iloc[-1, 0] - df_stock.iloc[0, 0]) / df_stock.iloc[0, 0]) *100, 2), '%')
print('CAGR:', round((((bt.MONEY + (df_join['종가'] * df_join['수량']).sum() - c) / first_monry) ** (1/1) - 1)*100, 2), '%')
print('-'*50)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:06<00:00, 18.52it/s]


--------------------------------------------------
기간:2020-05-01 00:00:00~2020-08-01 00:00:00
승률: 5.36 %
승리횟수:3
전체횟수:56
평가손익: 523299.0   평가손익율: 5.31
실현손익: -49379.000000000044
누적수익률 4.61 %
시장수익률: 18.68 %
CAGR: 4.61 %
--------------------------------------------------
